In [6]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

# Bayesian
# ==============================================================================
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
from sklearn.metrics import classification_report
import xgboost as xgb
from collections import Counter
import time
import optuna
from optuna.samplers import TPESampler
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

# Bayesian Multiobjetivo
# ==============================================================================
import pickle
import sys
import warnings

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Gradient boosting classifier
# ==============================================================================
from sklearn.ensemble import GradientBoostingClassifier



In [7]:
#carga los datos
tabla_final = pd.read_csv('Tabla_final_colab.csv', sep = ",")


## Se borran columnas irrelevantes ##
tabla_final = tabla_final.drop('FECHA_DATO', axis=1)
tabla_final = tabla_final.drop('FECHA_PROCESO', axis=1)
tabla_final = tabla_final.drop('ID_PROD', axis=1)
tabla_final = tabla_final.drop('FLAG_PREAP', axis=1)

## Definimos una tabla con los id de clientes
## Para luego poeder funtarla con las prob ##
## Si se queda se considera como parametro ##
tabla_con_id_cliente = tabla_final

tabla_final = tabla_final.drop('ID_CLIENTE', axis=1)
#visualizamos los datos
tabla_final

,CT_CTE,DEBITO_DIR,ID_EMPLEADO,ID_SEGMENTO_VALOR,ANTIGUEDAD,RENTA,EDAD,EDAD_PUNTAJE,ID_GENERO,ACEPTADO
0,1.0,0.0,0.0,2.0,6.0,87218.10,36.0,3.0,1.0,0.0
1,1.0,0.0,0.0,1.0,35.0,122179.11,23.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,35.0,119775.54,23.0,1.0,1.0,0.0
3,1.0,0.0,0.0,1.0,35.0,22220.04,24.0,1.0,1.0,0.0
4,1.0,0.0,0.0,1.0,35.0,295590.36,24.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
602555,0.0,0.0,0.0,2.0,62.0,97397.16,41.0,6.0,1.0,0.0
602556,0.0,0.0,0.0,2.0,7.0,168445.62,34.0,2.0,0.0,0.0
602557,0.0,0.0,0.0,1.0,6.0,53689.02,24.0,1.0,1.0,0.0
602558,0.0,0.0,0.0,2.0,229.0,64404.21,61.0,7.0,0.0,0.0


In [8]:
# Split the data into features (X) and target (y)
X = tabla_final.drop('ACEPTADO', axis=1)
y = tabla_final['ACEPTADO']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
# Bayesian Multiobjetivo
# ==============================================================================
def train(trial):
    
    max_depth = trial.suggest_int('max_depth', 5, X_train.shape[1]) #profundidad máxima del árbol
    n_estimators=trial.suggest_int('n_estimators',125, 200) #número de ramas en el árbol
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20) #mínimo de muestras en el nodo para que se pueda dividir
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 20) #mínimo de muestras en un nodo para que sea considerado una hoja
    min_impurity_decrease  = trial.suggest_float('min_impurity_decrease', 0, 1) #mínimo para supervisar si es necesario separar un nodo.
    
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", GradientBoostingClassifier(max_depth = max_depth, n_estimators = n_estimators, min_samples_split= min_samples_split, min_samples_leaf=min_samples_leaf, min_impurity_decrease=min_impurity_decrease))
    ])
    
    pipeline.fit(X_train, y_train)
    score = cross_val_score(pipeline, X_train, y_train, scoring='accuracy',
                            cv=4).mean()
    y_pred = pipeline.predict(X_test)
    precision = precision_score(y_test, y_pred)
    

    return score, precision


motpe_sampler = optuna.samplers.MOTPESampler()
motpe_experiment = optuna.create_study(sampler=motpe_sampler, directions="maximize")
motpe_experiment.optimize(train, n_trials=20)

/var/folders/cc/3mg7xtb906vcdzf2brdmqt500000gn/T/ipykernel_4459/3296854841.py:26: FutureWarning: MOTPESampler has been deprecated in v2.9.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.9.0.
  motpe_sampler = optuna.samplers.MOTPESampler()


ValueError: Please set either 'minimize' or 'maximize' to direction. You can also set the corresponding `StudyDirection` member.